In [17]:
def classify(url):
  !pip install cloudscraper
  !pip install beautifulsoup4
  !pip install --upgrade google-cloud-language

  import cloudscraper

  from bs4 import BeautifulSoup
  scraper = cloudscraper.create_scraper()
  headers = {'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121'}

  classifiers = {"Adult" : 10 , "Arts & Entertainment" : 12 , "Autos & Vehicles" : 2 , "Beauty & Fitness" : 15 , "Books & Literature" : 12 , "Business & Industrial" : 1 , "Computer and Electronics" : 18 , "Finance" : 18 , "Food & Drink" : 20 , "Games" : 12 , "Health" : 20 , "Hobbies & Leisure" : 10 , "Home & Garden" : 12,"Internet & Telecom" : 12,"Jobs & Education" : 15,"Law & Government" : 10,"News" : 8,"Online Communities" : 12,"People & Society" : 10,"Pets & Animals" : 15,"Real Estate" : 20,"Reference" : 12,"Science" : 12,"Sensitive Subjects" : 12,"Shopping" : 22,"Sports" : 10,"Travel & Transportation" : 12,"others" : 8, 'none': 0}

  try:
      r = scraper.get(url, headers = headers)

      soup = BeautifulSoup(r.text , 'html.parser')
      title = soup.find('title').text
      description = soup.find('meta', attrs={'name' : 'description'})

      if "content" in str(description):
        description = description.get("content")
      else:
        description = ""

      h1 = soup.find_all('h1')
      h1_all = ""
      for x in range (len(h1)):
        if x == len(h1)-1 :
          h1_all = h1_all + h1[x].text
        else:
          h1_all = h1_all + h1[x].text + ". "

      paragraphs_all = ""
      paragraphs = soup.find_all('p')
      for x in range (len(paragraphs)):
        if x == len(paragraphs)-1:
          paragraphs_all = paragraphs_all + paragraphs[x].text
        else:
          paragraphs_all = paragraphs_all + paragraphs[x].text + ". "


      h2 = soup.find_all('h2')
      h2_all = ""
      for x in range (len(h2)):
        if x == len(h2)-1:
          h2_all = h2_all + h2[x].text
        else:
          h2_all = h2_all + h2[x].text + ". "


      h3 = soup.find_all('h3')
      h3_all = ""
      for x in range (len(h3)):
        if x == len(h3)-1:
          h3_all = h3_all + h3[x].text
        else:
          h3_all = h3_all + h3[x].text + ". "

      allthecontent = str(title) + " " + str(description) + " " + str(h1_all) + " " + str(h2_all) + " " + str(h3_all) + " " + str(paragraphs_all)
      allthecontent = str(allthecontent)[0:999]

  except Exception as e:
    print(e)
    #return {'none':0}

  #title path to your Google NLP API credentials
  path_credential = "/content/gdrive/MyDrive/Colab Notebooks/WebsiteCategorization/keyfile.json"

  #We use the Google NLP API module to categorize the websites.
  #First, we need to provide the Google Application credentials.
  #We make the request to NLP API.
  #We print the assigned category and the confidence score.

  import os
  from google.cloud import language_v1
  from google.cloud import language
  from google.cloud.language_v1 import types
  from google.colab import drive
  #from google.cloud.language_v1.types import enums

  drive.mount('/content/gdrive')
  os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_credential
  # google nlp api's credentials to be created

  classifiers = {"Adult" : 10,"Arts & Entertainment" : 12,"Autos & Vehicles" : 2,"Beauty & Fitness" : 15,"Books & Literature" : 12,"Business & Industrial" : 1,"Computer and Electronics" : 18,"Finance" : 18,"Food & Drink" : 20,"Games" : 12,"Health" : 20,"Hobbies & Leisure" : 10,"Home & Garden" : 12,"Internet & Telecom" : 12,"Jobs & Education" : 15,"Law & Government" : 10,"News" : 8,"Online Communities" : 12,"People & Society" : 10,"Pets & Animals" : 15,"Real Estate" : 20,"Reference" : 12,"Science" : 12,"Sensitive Subjects" : 12,"Shopping" : 22,"Sports" : 10,"Travel & Transportation" : 12,"others" : 8,}

  try:
    text_content = str(allthecontent)
    client = language_v1.LanguageServiceClient()
    type_ = language_v1.Document.Type.PLAIN_TEXT
    #document = {"content" : text_content , "type": type_ , "language" : "en"}
    document = language_v1.Document(
          content=text_content,
          type=language_v1.Document.Type.PLAIN_TEXT,
          language="en"
      )
    encoding_type = types.EncodingType.UTF8
    # Make the API request
    response = client.classify_text(document=document)
    #response = client.classify_text(document)

    result = (response.categories[0].name).split('/')
    classification_response = result[1].strip()
    classification_confidence = int(float(response.categories[0].confidence)*100)

    if(classification_confidence < 15.00):
      classification_response = 'others'

    print(url + " categorized under " + classification_response + " with " + str(classification_confidence) + "% of confidence")
    #print(url + " categorized under " + response.categories[0].name + " with " + str(int(response.categories[0].confidence,3)*100) +"% of confidence")
    #print(url + " categorized under " + response.categories[0].name + " with " + str(int(response.categories[0].confidence, 3) * 100) + "% of confidence")
    return { classification_response : classifiers[classification_response]}

  except Exception as e:
    print(e)
    #return {'none': 0}

In [18]:
print(classify("https://instagram.com"))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
https://instagram.com categorized under Online Communities with 91% of confidence
{'Online Communities': 12}


In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import json
import cloudscraper
from bs4 import BeautifulSoup
from google.cloud import language_v1
from google.cloud.language_v1 import types

class RequestHandler(BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        payload = self.rfile.read(content_length)
        data = json.loads(payload.decode('utf-8'))

        try:
            url = data.get('url')

            # Scraping code
            scraper = cloudscraper.create_scraper()
            headers = {'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121'}
            r = scraper.get(url, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')

            # Extracting content from soup
            allthecontent = get_website_content(soup)

            # Google NLP API classification
            classification_response, classification_confidence = classify_content(allthecontent)

            result = {
                'url': url,
                'category': classification_response,
                'confidence': classification_confidence
            }

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(result).encode('utf-8'))

        except Exception as e:
            self.send_response(500)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'error': str(e)}).encode('utf-8'))

def get_website_content(soup):
    # Your code to extract content from soup
    # Example: Extracting title and paragraphs
    title = soup.find('title').text
    paragraphs = soup.find_all('p')
    paragraphs_all = " ".join([p.text for p in paragraphs])
    return f"{title} {paragraphs_all}"

def classify_content(allthecontent):
    # Your code for Google NLP API classification
    client = language_v1.LanguageServiceClient()
    type_ = language_v1.Document.Type.PLAIN_TEXT
    document = language_v1.Document(
        content=allthecontent,
        type=type_,
        language="en"
    )
    encoding_type = types.EncodingType.UTF8
    response = client.classify_text(document=document)

    # Extracting category and confidence
    result = (response.categories[0].name).split('/')
    classification_response = result[1].strip()
    classification_confidence = int(float(response.categories[0].confidence) * 100)

    if classification_confidence < 15.00:
        classification_response = 'others'

    return classification_response, classification_confidence

if __name__ == '__main__':
    server_address = ('', 8000)
    httpd = HTTPServer(server_address, RequestHandler)
    print('Starting server on port 8000...')
    httpd.serve_forever()


In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import json
import cloudscraper
import os
from bs4 import BeautifulSoup
from google.cloud import language_v1
from google.cloud import language
from google.cloud.language_v1 import types
from google.colab import drive

class RequestHandler(BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        payload = self.rfile.read(content_length)
        data = json.loads(payload.decode('utf-8'))

        try:
            url = data.get('url')

            # code for scraping
            scraper = cloudscraper.create_scraper()
            headers = {'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121'}

            r = scraper.get(url, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')

            # Your existing code for classification
            allthecontent = get_website_content(soup)

            # Your existing code for Google NLP API classification
            classification_response, points = classify_content(allthecontent)

            result = {
                'url': url,
                'category': classification_response,
                'points': points
            }

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(result).encode('utf-8'))

        except Exception as e:
            self.send_response(500)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'error': str(e)}).encode('utf-8'))

def get_website_content(soup):
    title = soup.find('title').text
    description = soup.find('meta', attrs={'name':'description'})

    if "content" in str(description):
      description = description.get("content")
    else:
      description = ""

    h1 = soup.find_all('h1')
    h1_all = ""
    for x in range (len(h1)):
      if x == len(h1)-1 :
        h1_all = h1_all + h1[x].text
      else:
        h1_all = h1_all + h1[x].text + ". "

    paragraphs_all = ""
    paragraphs = soup.find_all('p')
    for x in range (len(paragraphs)):
      if x == len(paragraphs)-1:
        paragraphs_all = paragraphs_all + paragraphs[x].text
      else:
        paragraphs_all = paragraphs_all + paragraphs[x].text + ". "


    h2 = soup.find_all('h2')
    h2_all = ""
    for x in range (len(h2)):
      if x == len(h2)-1:
        h2_all = h2_all + h2[x].text
      else:
        h2_all = h2_all + h2[x].text + ". "


    h3 = soup.find_all('h3')
    h3_all = ""
    for x in range (len(h3)):
      if x == len(h3)-1:
        h3_all = h3_all + h3[x].text
      else:
        h3_all = h3_all + h3[x].text + ". "

    allthecontent = str(title) + " " + str(description) + " " + str(h1_all) + " " + str(h2_all) + " " + str(h3_all) + " " + str(paragraphs_all)
    allthecontent = str(allthecontent)[0:999]

def classify_content(allthecontent):
    path_credential = "/content/gdrive/MyDrive/Colab Notebooks/WebsiteCategorization/keyfile.json"
    drive.mount('/content/gdrive')
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_credential
    classifiers = {"Adult" : 10,"Arts & Entertainment" : 12,"Autos & Vehicles" : 2,"Beauty & Fitness" : 15,"Books & Literature" : 12,"Business & Industrial" : 1,"Computer and Electronics" : 18,"Finance" : 18,"Food & Drink" : 20,"Games" : 12,"Health" : 20,"Hobbies & Leisure" : 10,"Home & Garden" : 12,"Internet & Telecom" : 12,"Jobs & Education" : 15,"Law & Government" : 10,"News" : 8,"Online Communities" : 12,"People & Society" : 10,"Pets & Animals" : 15,"Real Estate" : 20,"Reference" : 12,"Science" : 12,"Sensitive Subjects" : 12,"Shopping" : 22,"Sports" : 10,"Travel & Transportation" : 12,"others" : 8,}

    try:
      text_content = str(allthecontent)
      client = language_v1.LanguageServiceClient()
      type_ = language_v1.Document.Type.PLAIN_TEXT
      #document = {"content" : text_content , "type": type_ , "language" : "en"}
      document = language_v1.Document(
            content=text_content,
            type=language_v1.Document.Type.PLAIN_TEXT,
            language="en"
        )
      encoding_type = types.EncodingType.UTF8
      # Make the API request
      response = client.classify_text(document=document)
      #response = client.classify_text(document)

      print(" categorized under " + response.categories[0].name + " with " + str(int(float(response.categories[0].confidence)*100)) + "% of confidence")
      #print(url + " categorized under " + response.categories[0].name + " with " + str(int(response.categories[0].confidence,3)*100) +"% of confidence")
      #print(url + " categorized under " + response.categories[0].name + " with " + str(int(response.categories[0].confidence, 3) * 100) + "% of confidence")

      # Extracting category and confidence
      result = (response.categories[0].name).split('/')
      classification_response = result[1].strip()
      classification_confidence = int(float(response.categories[0].confidence) * 100)

      if classification_confidence < 15.00:
          classification_response = 'others'

      return classification_response, classifiers[classification_response]

    except Exception as e:
      print(e)
      return 'none', 0

if __name__ == '__main__':
    server_address = ('', 8000)
    httpd = HTTPServer(server_address, RequestHandler)
    print('Starting server on port 8000...')
    #httpd.serve_forever()


Starting server on port 8000...
